# 1. Protein data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/proteinmodel30', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel30');
models = s.models;


In [9]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans BCM
BCM: SMSE 25.59297575, MSLL 3.4002, NLPD 6.6370
Processing protein kmeans PoE
PoE: SMSE 26.46774598, MSLL 4.0657, NLPD 7.3025


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [10]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans RBCM
RBCM: SMSE 25.05414455, MSLL 5.9183, NLPD 9.1551


### 1.1.2 Grid q

In [11]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 25.25252127, MSLL 278.0914, NLPD 281.3282 - q   0.20
TERBCM: SMSE 25.20630443, MSLL 95.3816, NLPD 98.6184 - q   0.40
TERBCM: SMSE 25.15680788, MSLL 37.3626, NLPD 40.5994 - q   0.60
TERBCM: SMSE 25.10522339, MSLL 15.0932, NLPD 18.3299 - q   0.80
TERBCM: SMSE 25.05414455, MSLL 5.9183, NLPD 9.1551 - q   1.00
TERBCM: SMSE 25.00937676, MSLL 2.0586, NLPD 5.2953 - q   1.20
TERBCM: SMSE 24.98445947, MSLL 0.4826, NLPD 3.7194 - q   1.40
TERBCM: SMSE 25.01368316, MSLL -0.0768, NLPD 3.1600 - q   1.60
TERBCM: SMSE 25.19097554, MSLL -0.1754, NLPD 3.0614 - q   1.80
TERBCM: SMSE 25.76376686, MSLL -0.0671, NLPD 3.1696 - q   2.00
TERBCM: SMSE 27.31452862, MSLL 0.1255, NLPD 3.3623 - q   2.20
TERBCM: SMSE 30.95800606, MSLL 0.3384, NLPD 3.5751 - q   2.40
TERBCM: SMSE 38.06848829, MSLL 0.5345, NLPD 3.7713 - q   2.60
TERBCM: SMSE 49.00374752, MSLL 0.6935, NLPD 3.9302 - q   2.80
TERBCM: SMSE 61.93497004, MSLL 0.8086, NLPD 4.0454 - q   3.00
TERBCM: SMSE 73.90063746

### 1.1.3 Find single q

In [12]:
criterion = 'TERBCM';

In [13]:
printf('Optimizing q\r\n');
q1 = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.8)
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q1) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.8000
q1 =  1.8084
TERBCM: SMSE 25.20450251, MSLL -0.1738, NLPD 3.0629


### 1.1.4 Find multi q

In [9]:
criterion = 'TERBCM';

In [17]:
printf('Optimizing q\r\n');
iqs = 1.80 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.8071
   1.8039
   1.8084
   1.8054
   1.8078
   1.8091
   1.8047
   1.8004
   1.8005
   1.8039
   1.8025
   1.8063
   1.8087
   1.8040
   1.8059
   1.8063
   1.8052
   1.8040
   1.8026
   1.8094
   1.8041
   1.8024
   1.8046
   1.8016
   1.8083
   1.8058
   1.8098
   1.8085
   1.8017
   1.8048

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   1.6521
   1.7810
   1.7499
   1.7862
   1.7812
   1.7610
   1.7774
   1.8002
   1.7648
   1.7515
   1.7682
   1.8018
   1.7537
   1.7615
   1.7405
   1.7557
   1.7909
   1.7744
   1.7698
   1.7866
   1.8024
   1.7983
   1.7919
   1.8016
   1.7571
   1.7648
   1.7718
   1.7843
   1.7991
   1.7970

TERBCM: SMSE 24.98386136, MSLL -0.1778, NLPD 3.0590
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   1.6703
   1.7849
   1.7548
   1.7874
   1.7827
   1.7658
   1.7823
   1.8002
   1.7679
   1.7573
   1.7739
   1.8024
   1.7606
   1.7652
   1.7474
   1.7590
   1.7924
   1.7759
   1.7721
   1.788

In [18]:
printf('Optimizing q\r\n');
iqs = 1.4 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.4085
   1.4061
   1.4049
   1.4059
   1.4017
   1.4027
   1.4080
   1.4087
   1.4031
   1.4004
   1.4073
   1.4095
   1.4089
   1.4039
   1.4052
   1.4003
   1.4036
   1.4089
   1.4001
   1.4088
   1.4083
   1.4058
   1.4040
   1.4012
   1.4042
   1.4052
   1.4048
   1.4096
   1.4001
   1.4049

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   0.52328
   1.30312
   0.89697
   1.30424
   1.24761
   1.10386
   1.27540
   1.40805
   1.19288
   1.01640
   1.22576
   1.39162
   1.03153
   1.15513
   0.83459
   1.00153
   1.33466
   1.23653
   1.20430
   1.28736
   1.40050
   1.38915
   1.33825
   1.40115
   1.04065
   1.15106
   1.20841
   1.27650
   1.38789
   1.36736

TERBCM: SMSE 24.64963770, MSLL 8.8375, NLPD 12.0743
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   1.0994
   1.3702
   1.2215
   1.3387
   1.3149
   1.3060
   1.3822
   1.4080
   1.2935
   1.2754
   1.3423
   1.3978
   1.2813
   1.3085
   1.2269
   1.2398
   1.358

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [12]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 23.69895202, MSLL -0.2228, NLPD 3.0049


### 1.2.2 Grid q

In [13]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 24.26368176, MSLL 0.1533, NLPD 3.3810 - q   0.20
TEGRBCM: SMSE 23.14018847, MSLL -0.1376, NLPD 3.0900 - q   0.40
TEGRBCM: SMSE 23.18461282, MSLL -0.2082, NLPD 3.0194 - q   0.60
TEGRBCM: SMSE 23.43707393, MSLL -0.2237, NLPD 3.0039 - q   0.80
TEGRBCM: SMSE 23.69895202, MSLL -0.2228, NLPD 3.0049 - q   1.00
TEGRBCM: SMSE 23.92349921, MSLL -0.2175, NLPD 3.0101 - q   1.20
TEGRBCM: SMSE 24.10781720, MSLL -0.2121, NLPD 3.0155 - q   1.40
TEGRBCM: SMSE 24.25983745, MSLL -0.2078, NLPD 3.0199 - q   1.60
TEGRBCM: SMSE 24.38537411, MSLL -0.2047, NLPD 3.0230 - q   1.80
TEGRBCM: SMSE 24.48572389, MSLL -0.2027, NLPD 3.0250 - q   2.00
TEGRBCM: SMSE 24.56077510, MSLL -0.2015, NLPD 3.0262 - q   2.20
TEGRBCM: SMSE 24.61227329, MSLL -0.2007, NLPD 3.0269 - q   2.40
TEGRBCM: SMSE 24.64473948, MSLL -0.2003, NLPD 3.0273 - q   2.60
TEGRBCM: SMSE 24.66392463, MSLL -0.2001, NLPD 3.0276 - q   2.80
TEGRBCM: SMSE 24.67482038, MSLL -0.2000, NLPD 3.0277 - q   3.00
TEGRBCM

### 1.2.3 Find single q

In [14]:
criterion = 'TEGRBCM';

In [15]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.8);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.8000
Optimizing q: remain iter - 1000, q - 0.8000


In [16]:
q

q =  0.80455


In [17]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [7]:
criterion = 'TEGRBCM';

In [8]:
printf('Optimizing q\r\n');
iqs = 0.8 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-1, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   0.80995
   0.80852
   0.80300
   0.80883
   0.80102
   0.80012
   0.80932
   0.80528
   0.80210
   0.80230
   0.80640
   0.80351
   0.80719
   0.80893
   0.80201
   0.80593
   0.80409
   0.80525
   0.80896
   0.80789
   0.80426
   0.80492
   0.80658
   0.80851
   0.80062
   0.80435
   0.80173
   0.80902
   0.80881
   0.80145

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   0.80995
   0.80852
   0.80214
   0.80774
   0.80136
   0.80079
   0.80941
   0.80524
   0.80141
   0.80083
   0.80691
   0.80355
   0.80653
   0.80887
   0.80220
   0.80512
   0.80403
   0.80439
   0.80886
   0.80686
   0.80338
   0.80424
   0.80646
   0.80851
   0.79929
   0.80410
   0.80209
   0.80858
   0.80797
   0.80242

TEGRBCM: SMSE 23.443

In [9]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-1, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   0.40974
   0.40401
   0.40692
   0.40163
   0.40739
   0.40947
   0.40213
   0.40445
   0.40738
   0.40881
   0.40087
   0.40448
   0.40027
   0.40883
   0.40189
   0.40796
   0.40576
   0.40573
   0.40257
   0.40077
   0.40250
   0.40097
   0.40217
   0.40731
   0.40666
   0.40793
   0.40357
   0.40317
   0.40761
   0.40335

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   0.40974
   0.40401
   0.41255
   0.40744
   0.41443
   0.41517
   0.40556
   0.40530
   0.41159
   0.41004
   0.40373
   0.40705
   0.40267
   0.40951
   0.40592
   0.41248
   0.41983
   0.41269
   0.40230
   0.40655
   0.40305
   0.40264
   0.40556
   0.40732
   0.40574
   0.41567
   0.40627
   0.40800
   0.41232
   0.41251

TEGRBCM: SMSE 23.122

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [10]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GPoE
GPoE: SMSE 24.31338867, MSLL 6.0500, NLPD 9.2776


### 1.3.2 Grid q

In [11]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGPoE
TEGPoE: SMSE 24.53083990, MSLL 285.0609, NLPD 288.2885 - q   0.20
TEGPoE: SMSE 24.48269765, MSLL 97.7122, NLPD 100.9398 - q   0.40
TEGPoE: SMSE 24.43024829, MSLL 38.2540, NLPD 41.4817 - q   0.60
TEGPoE: SMSE 24.37370008, MSLL 15.4442, NLPD 18.6719 - q   0.80
TEGPoE: SMSE 24.31338867, MSLL 6.0500, NLPD 9.2776 - q   1.00
TEGPoE: SMSE 24.24976690, MSLL 2.0979, NLPD 5.3255 - q   1.20
TEGPoE: SMSE 24.18338468, MSLL 0.4837, NLPD 3.7114 - q   1.40
TEGPoE: SMSE 24.11486107, MSLL -0.0884, NLPD 3.1393 - q   1.60
TEGPoE: SMSE 24.04485156, MSLL -0.1842, NLPD 3.0434 - q   1.80
TEGPoE: SMSE 23.97401450, MSLL -0.0590, NLPD 3.1687 - q   2.00
TEGPoE: SMSE 23.90298036, MSLL 0.1696, NLPD 3.3972 - q   2.20
TEGPoE: SMSE 23.83232639, MSLL 0.4458, NLPD 3.6735 - q   2.40
TEGPoE: SMSE 23.76255871, MSLL 0.7433, NLPD 3.9710 - q   2.60
TEGPoE: SMSE 23.69410194, MSLL 1.0492, NLPD 4.2769 - q   2.80
TEGPoE: SMSE 23.64887656, MSLL 1.3574, NLPD 4.5850 - q   3.00
TEGPoE: SMSE 23.5968435

### 1.3.3 Find single q

In [18]:
criterion = 'TEGPoE';

In [21]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.8);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.8000


In [22]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [23]:
printf('Optimizing q\r\n');
iqs = 1.80 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.8018
   1.8083
   1.8074
   1.8065
   1.8060
   1.8092
   1.8037
   1.8036
   1.8030
   1.8093
   1.8011
   1.8005
   1.8075
   1.8008
   1.8020
   1.8042
   1.8070
   1.8064
   1.8058
   1.8067
   1.8064
   1.8046
   1.8053
   1.8033
   1.8000
   1.8038
   1.8094
   1.8074
   1.8039
   1.8010

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   1.9626
   1.8950
   1.9140
   1.8513
   1.8734
   1.9326
   1.9270
   1.8051
   1.8903
   1.9334
   1.9092
   1.8203
   1.9302
   1.9105
   1.9176
   1.9087
   1.8449
   1.8803
   1.8697
   1.8567
   1.8137
   1.8192
   1.8366
   1.8034
   1.9236
   1.8970
   1.9334
   1.8630
   1.8147
   1.8254

TEGPoE: SMSE 24.08220228, MSLL -0.1365, NLPD 3.0912
ld =  1
Start preprocessing.

In [24]:
printf('Optimizing q\r\n');
iqs = 3.6 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   3.6010
   3.6085
   3.6035
   3.6010
   3.6030
   3.6023
   3.6052
   3.6091
   3.6036
   3.6007
   3.6095
   3.6050
   3.6044
   3.6097
   3.6004
   3.6007
   3.6047
   3.6087
   3.6009
   3.6008
   3.6034
   3.6019
   3.6060
   3.6025
   3.6035
   3.6022
   3.6072
   3.6001
   3.6059
   3.6067

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   3.6016
   3.6087
   3.6038
   3.6011
   3.6031
   3.6027
   3.6056
   3.6091
   3.6038
   3.6011
   3.6098
   3.6051
   3.6048
   3.6100
   3.6007
   3.6010
   3.6048
   3.6089
   3.6011
   3.6009
   3.6034
   3.6019
   3.6060
   3.6025
   3.6039
   3.6024
   3.6076
   3.6002
   3.6059
   3.6067

TEGPoE: SMSE 23.51545080, MSLL 2.2790, NLPD 5.5067
ld =  1
Start preprocessing..